In [1]:
import mne
import numpy as np
from scipy.io import loadmat

In [2]:
# Load the .mat file
data = loadmat('/Dev/stroke-rehab-data-analysis/data/stroke-rehab/P1_post_test.mat')

# Display the keys in the loaded data
print(data.keys())

channels = ['FC3', 'FCz', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'Pz']
fs = float(data['fs'].squeeze())
info = mne.create_info(ch_names=channels, sfreq=fs, ch_types=['eeg' for _ in channels])
triggers = data['trig'].ravel() 

dict_keys(['__header__', '__version__', '__globals__', 'fs', 'trig', 'y'])


In [3]:
raw = mne.io.RawArray(data['y'].T, info)

Creating RawArray with float64 data, n_channels=16, n_times=194088
    Range : 0 ... 194087 =      0.000 ...   758.152 secs
Ready.


In [4]:
#raw.plot(duration=10, scalings=1e2, n_channels=3)

## Annotations

In [5]:
triggers.shape

(194088,)

In [6]:
# triggers: 1D array of length n_samples with values {0, 1, -1}
# fs: sampling frequency

# 1) find sample‐indices where triggers change
padded = np.r_[0, triggers, 0]       # pad with zeros at ends
diffs  = np.diff(padded)             # rises: +1 or -1; falls: -1 or +1

# 2) onsets where diff != 0
onset_samples  = np.where(diffs != 0)[0]
# they come in pairs [start, end, start, end, …]
onsets = onset_samples[::2]          # even indices: starts
offsets = onset_samples[1::2]        # odd indices: ends

# 3) map each onset to its trigger value
#    we look at the original trigger at the sample of the onset
values = triggers[onsets]

# 4) convert to seconds
onset_times = onsets / fs
durations   = (offsets - onsets) / fs

# 5) text descriptions
descriptions = ['left' if val==1 else 'right' for val in values]


In [7]:
# create the Annotations object
annot = mne.Annotations(onset=onset_times,
                        duration=durations,
                        description=descriptions)

# attach to your Raw
raw.set_annotations(annot)


<RawArray | 16 x 194088 (758.2 s), ~23.7 MiB, data loaded>

In [8]:
events, event_id = mne.events_from_annotations(
    raw,
    event_id={'left': 1, 'right': 2},  # map names → integer codes
)
# shift all events forward
events[:, 0] += int(2 * fs)

Used Annotations descriptions: [np.str_('left'), np.str_('right')]


## Epochs

In [9]:
# epoching
epochs = mne.Epochs(raw, events, event_id=event_id)

Not setting metadata
80 matching events found
Setting baseline interval to [-0.19921875, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
